In [1]:
#notebook based on zflemings:https://nbviewer.jupyter.org/github/zflamig/dask-era5/blob/main/notebook/era5_fargate_dask.ipynb
import matplotlib.pyplot as plt
import xarray as xr
import fsspec
import dask
import s3fs
import numpy as np
import gcsfs
import fsspec
import xarray as xr

import dask
from dask.distributed import performance_report, Client, progress

xr.set_options(display_style="html")  #display dataset nicely 

#lib for dask gateway
from dask_gateway import Gateway
from dask.distributed import Client
from dask import delayed

In [2]:
gateway = Gateway()
cluster = gateway.new_cluster()
cluster.adapt(minimum=1, maximum=75)
client = Client(cluster)
cluster

In [3]:
#zac code
def fix_accum_var_dims(ds, var):
    # Some varibles like precip have extra time bounds varibles, we drop them here to allow merging with other variables
    
    # Select variable of interest (drops dims that are not linked to current variable)
    ds = ds[[var]]  

    if var in ['air_temperature_at_2_metres',
               'dew_point_temperature_at_2_metres',
               'air_pressure_at_mean_sea_level',
               'northward_wind_at_10_metres',
               'eastward_wind_at_10_metres']:
        
        ds = ds.rename({'time0':'valid_time_end_utc'})
        
    elif var in ['precipitation_amount_1hour_Accumulation',
                 'integral_wrt_time_of_surface_direct_downwelling_shortwave_flux_in_air_1hour_Accumulation']:
        
        ds = ds.rename({'time1':'valid_time_end_utc'})
        
    else:
        print("Warning, Haven't seen {var} varible yet! Time renaming might not work.".format(var=var))
        
    return ds

@dask.delayed
def s3open(path):
    fs = s3fs.S3FileSystem(anon=True, default_fill_cache=False, 
                           config_kwargs = {'max_pool_connections': 20})
    return s3fs.S3Map(path, s3=fs)


def open_era5_range(start_year, end_year, variables):
    ''' Opens ERA5 monthly Zarr files in S3, given a start and end year (all months loaded) and a list of variables'''
    
    
    file_pattern = 'era5-pds/zarr/{year}/{month}/data/{var}.zarr/'
    
    years = list(np.arange(start_year, end_year+1, 1))
    months = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]
    
    l = []
    for var in variables:
        print(var)
        
        # Get files
        files_mapper = [s3open(file_pattern.format(year=year, month=month, var=var)) for year in years for month in months]
        
        # Look up correct time dimension by variable name
        if var in ['precipitation_amount_1hour_Accumulation']:
            concat_dim='time1'
        else:
            concat_dim='time0'
            
        # Lazy load
        ds = xr.open_mfdataset(files_mapper, engine='zarr', 
                               concat_dim=concat_dim, combine='nested', 
                               coords='minimal', compat='override', parallel=True)
        
        # Fix dimension names
        ds = fix_accum_var_dims(ds, var)
        l.append(ds)
        
    ds_out = xr.merge(l)
    
    return ds_out



In [4]:
%%time
ds_uwnd = open_era5_range(1982, 2020, ["eastward_wind_at_10_metres"])

eastward_wind_at_10_metres
CPU times: user 10 s, sys: 766 ms, total: 10.8 s
Wall time: 11min 44s


In [5]:
%%time
ds_vwnd = open_era5_range(1982, 2020, ["northward_wind_at_10_metres"])

northward_wind_at_10_metres
CPU times: user 9.8 s, sys: 668 ms, total: 10.5 s
Wall time: 10min 28s


In [6]:
%%time
ds_sst = open_era5_range(1982, 2020, ["sea_surface_temperature"])
ds_sst=ds_sst.rename({'time0':'time'})
ds_sst

sea_surface_temperature
Warning, Haven't seen sea_surface_temperature varible yet! Time renaming might not work.
CPU times: user 10.1 s, sys: 600 ms, total: 10.7 s
Wall time: 9min 12s


In [7]:
ds_wspd = np.sqrt(ds_uwnd.eastward_wind_at_10_metres**2+ds_vwnd.northward_wind_at_10_metres**2)
ds_wspd=ds_wspd.rename({'valid_time_end_utc':'time'})
ds_wspd

In [20]:
from scipy.ndimage import gaussian_filter
from scipy.stats.stats import pearsonr
from scipy import signal

In [ ]:
#SST
#remove mean, seasonal cycle, trend before analysis
clim = ds_sst.groupby('time.dayofyear').mean('time',keep_attrs=True,skipna=False)
ds_tem = ds_sst.groupby('time.dayofyear')-clim
#detrending doesn't work with nan so fill with nan but will have to mask at end and pixels with nan in timeseries
tem = ds_tem.fillna(0)
ds_detrended_sst = signal.detrend(tem.sea_surface_temperature,axis=0)

In [ ]:
ds_new_tem.analysed_sst.isel(lon=720,lat=310).plot()
plt.plot(ds_new_tem.time,ds_detrended_sst[:,310,720])

In [ ]:
#WSPD
#remove mean, seasonal cycle, trend before analysis
clim = ds_ws[d.groupby('time.dayofyear').mean('time',keep_attrs=True,skipna=False)
ds_new_tem = ds_wspd.groupby('time.dayofyear')-clim
#detrending doesn't work with nan so fill with nan but will have to mask at end and pixels with nan in timeseries
tem = ds_new_tem.fillna(0)
ds_detrended_wnd = signal.detrend(tem,axis=0)

In [ ]:
ds_new_tem.isel(lon=720,lat=310).plot()
plt.plot(ds_new_tem.time,ds_detrended_wnd[:,310,720])

In [ ]:
corr = multi_apply_along_axis(pearsonr, 0, [ds_detrended_sst,ds_detrended_wnd])
print(corr.shape)
print(type(corr))
#not sure why by 1440 is blank
acorr = corr[0,:,:]
acorr2 = np.concatenate([acorr[:,720:1439],acorr[:,:1439],acorr[:,:720]],axis=1)
#plt.imshow(acorr[:,:1439],vmin=-.1,vmax=.1,cmap='RdBu')
#plt.imshow(acorr2,vmin=-.1,vmax=.1,cmap='viridis')
data = acorr2
data = np.nan_to_num(data,0)
lowpass = ndimage.gaussian_filter(data, 40)
gauss_highpass = data - lowpass



In [ ]:
plt.imshow(lowpass,vmin=-.1,vmax=.1,cmap='RdBu')
plt.imshow(gauss_highpass,vmin=-.1,vmax=.1,cmap='RdBu')